# Clustering de documents de la décénie 1960–1969

In [ ]:
import collections
import os
import string
import sys

import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine

from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
import nltk

nltk.download('punkt')

In [ ]:
data_path = "../data/txt/"

## Décénie 1960–1969

In [ ]:
DECADE = '1960'

In [ ]:
files = [f for f in sorted(os.listdir(data_path)) if f"_{DECADE[:-1]}" in f]

In [ ]:
# Exemple de fichiers
files[:5]

In [ ]:
texts = [open(data_path + f, "r", encoding="utf-8").read() for f in files]

In [ ]:
# Exemple de textes
texts[0][:400]

In [ ]:
# Création d'une fonction de pré-traitement
def preprocessing(text, stem=True):
    """ Tokenize text and remove punctuation """
    text = text.translate(string.punctuation)
    tokens = word_tokenize(text)
    return tokens

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer=preprocessing,
    stop_words=stopwords.words('french'),
    max_df=0.5,
    min_df=0.1,
    lowercase=True)

In [ ]:
tfidf_vectors = vectorizer.fit_transform(texts)

In [ ]:
# Détail de la matrice
tfidf_vectors

In [ ]:
pd.Series(
    tfidf_vectors[0].toarray()[0],
    index=vectorizer.get_feature_names_out()
    ).sort_values(ascending=False)

In [ ]:
tfidf_array = tfidf_vectors.toarray()

## Nombre de clusters définis à 5

In [ ]:
N_CLUSTERS = 5

In [ ]:
km_model = KMeans(n_clusters=N_CLUSTERS)

In [ ]:
clusters = km_model.fit_predict(tfidf_vectors)

In [ ]:
clustering = collections.defaultdict(list)

for idx, label in enumerate(clusters):
    clustering[label].append(files[idx])

In [ ]:
pprint(dict(clustering))

In [ ]:
pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(tfidf_vectors.toarray())

In [ ]:
reduced_vectors[:10]

In [ ]:
x_axis = reduced_vectors[:, 0]
y_axis = reduced_vectors[:, 1]

plt.figure(figsize=(10,10))
scatter = plt.scatter(x_axis, y_axis, s=100, c=clusters)

# Ajouter les centroïdes
centroids = pca.transform(km_model.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1],  marker = "x", s=100, linewidths = 2, color='black')

# Ajouter la légende
plt.legend(handles=scatter.legend_elements()[0], labels=set(clusters), title="Clusters")

## Extraction des documents de chaque cluster 

In [ ]:

# Récupération des étiquettes de cluster pour chaque document
labels = km_model.labels_

# Créer un dictionnaire pour stocker les documents de chaque cluster
clusters_documents = {}
for i, label in enumerate(labels):
    if label not in clusters_documents:
        clusters_documents[label] = []
    clusters_documents[label].append(i)  # Ajouter l'index du document au cluster correspondant

for cluster_label, documents_indices in clusters_documents.items():
    print(f"Cluster {cluster_label}:")
    for doc_index in documents_indices:
        print(texts[doc_index])  # Afficher le document correspondant à l'indice
    print("\n")  # Ajouter un saut de ligne entre les clusters


## Analyse : mots clés des clusters

In [ ]:
import os
import yake
from collections import Counter
from nltk.corpus import stopwords
import re

In [ ]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

## Récupérer les mots clés pour chaque document de chaque clusters 

In [ ]:

# Parcours chaque cluster
for cluster_label, documents_indices in clusters_documents.items():
    print(f"Mots-clés pour le cluster {cluster_label}:")
    for doc_index in documents_indices:
        text = texts[doc_index]  # Récupérer le texte du document
        keywords = kw_extractor.extract_keywords(text)  # Extraire les mots-clés du texte
        kept = []
        for kw, score in keywords:
            words = kw.split()
            if len(words) == 2:
                kept.append(kw)
        print(f"Document {doc_index} mentions these keywords: {', '.join(kept)}...")
    print("\n")  # Saut de ligne entre les clusters

## Regroupement des mots clés les plus fréquents par clusters 

In [ ]:

# Charger les mots vides de la langue correspondante 
stop_words = set(stopwords.words("french"))

# Ajouter des mots à la liste des stop words
additional_stop_words = ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
                        "ils", "elles", "il", "elle", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
                        "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
                        "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
                        "van", "het", "autre", "jusqu", "ville", "rossel", "dem", "si", "car", "autant",
                        "toute", "très", "bien", "aucun", "comme", "celui", "chaque", "plusieurs",
                        "toutes", "trop", "aucune", "parce", "quelques", "quel", "quelle", "quels",
                        "quelles", "lorsqu", "lorsque", "dès", "dans", "leurs", "peu", "toute", "toutes",
                        "DÈS", "près", "quart", "part", "ETC", "demi", "bas", "grand", "demande", "vers",
                        "avant", "vers","aussi","ans","leurs","très"]

# Parcours chaque cluster
for cluster_label, documents_indices in clusters_documents.items():
    print(f"Mots-clés fréquents pour le cluster {cluster_label}:")
    
    # Collecter tous les mots non vides, non numériques et ne contenant pas une seule lettre de tous les documents du cluster
    all_words = []
    for doc_index in documents_indices:
        text = texts[doc_index]
        words = re.findall(r'\b[A-Za-zÀ-ÿ]+\b', text)  # Trouver les mots alphabétiques
        # Filtrer les mots vides, les mots qui sont des chiffres et les mots d'une seule lettre
        words = [word for word in words if word.lower() not in stop_words and not word.isdigit() and len(word) > 1]
        all_words.extend(words)  # Ajouter les mots satisfaisant les critères à la liste
    
    # Calculer la fréquence des mots non vides, non numériques et ne contenant pas une seule lettre
    word_freq = Counter(all_words)
    
    # Sélectionner les mots les plus fréquents
    top_keywords = word_freq.most_common(200)
    
    # Afficher les mots-clés fréquents pour ce cluster
    for keyword, frequency in top_keywords:
        print(f"Mot-clé: {keyword}, Fréquence: {frequency}")
    print("\n")  # Saut de ligne entre les clusters

## Analyse : nuage de mots

In [ ]:
# Parcours chaque cluster
for cluster_label, documents_indices in clusters_documents.items():
    print(f"Nuage de mots pour le cluster {cluster_label}:")
    
    
    # Créer une chaîne de mots pour le nuage de mots
    wordcloud_text = ' '.join([keyword for keyword, _ in top_keywords])
    
    # Créer et afficher le nuage de mots
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(wordcloud_text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"Nuage de mots pour le cluster {cluster_label}")
    plt.show()

## Entraînement d'un modèle word2vec

In [ ]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

In [ ]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [ ]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

In [ ]:
bigram_phrases = Phrases(sentences)

In [ ]:
type(bigram_phrases.vocab)

In [ ]:
len(bigram_phrases.vocab.keys())

In [ ]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

In [ ]:
bigram_phrases.vocab[key_]

In [ ]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

In [ ]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [ ]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

In [ ]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

## Adaptation des paramètres window et min_count

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=10, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

In [ ]:
outfile = f"../data/newspapers.model"
model.save(outfile)

In [ ]:
model = Word2Vec.load("../data/newspapers.model")

In [ ]:
model.wv["paris"]

## Calculer la similarité entre deux termes

In [ ]:
model.wv.similarity("paris", "capitale")

## Chercher les mots les plus proches d'un terme donné

In [ ]:
model.wv.most_similar("paris", topn=10)

In [ ]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))
